In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from langchain_ollama import OllamaLLM
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# # 최신버전 1.x 대응 임포트 NOTE [20260114@terry007x] langchain 1.2.3 에서 module import 안 됨... AgentType을 String 값으로 지정 agent_type="zero-shot-react-description",
# try:
#     from langchain.agents.agent_types import AgentType
# except ImportError:
#     # 혹시 모를 하위 호환성 위한 예외 처리... 미치겠다...
#     from langchain.agents import AgentType

# 1. 시각화 설정
%matplotlib inline

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False
sns.set_theme(style="whitegrid", font='NanumGothic') # Seaborn 테마 설정

# 2. 분석할 CSV 파일 로드 
file_path = "./input/tpss_bcycl_od_statnhm_20260104.csv" 
df = pd.read_csv(file_path, encoding='cp949')

print("데이터 로드 완료")
display(df.head()) 
print(len(df))

# 3. 모델 및 에이전트 설정
model_name = "mistral"  # "mistral"  # "mistral-nemo"  # "mistral-small"  # "mixtral"  # # 크기 순 오름차순
"""
mixtral: \ 문제
"""
llm = OllamaLLM(model=model_name, 
                temperature=0.0,
               )
llm_with_stop = llm.bind(stop=["Observation:", "Observation", "\nObservation"])  # 중지 시퀀스 설정
# 에이전트가 지켜야 할 철저한 규칙 정의
custom_prefix = """
You are a Python data analysis expert.
Your goal is to provide accurate Python code to analyze the dataframe `df`.
You must use the following format:

Thought: your reasoning
Action: python_repl_ast
Action Input: your python code (WITHOUT ANY MARKDOWN BACKTICKS OR ```python TAGS)
Observation: the result
... (repeat)
Final Answer: final report in Korean

CRITICAL: Never use ```python or ``` blocks in Action Input. Just write raw code.
If you have obtained the necessary statistical figures through the Observation, stop executing any further code and immediately write a Korean report using 'Final Answer:'. Generating the final answer takes priority over repetitive code execution.

[CRITICAL RULES FOR PYTHON CODE]
1. NEVER use backslashes (`\`) to escape underscores (`_`). 
   - Incorrect: df.groupby('시작\_대여소\_ID')
   - Correct: df.groupby('시작_대여소_ID')
2. The tool name is exactly `python_repl_ast`. Do not use `python\_repl\_ast`.
3. Provide RAW code only in Action Input. No markdown formatting inside Action Input.

[FATAL RULE]
- No conversational filler.
- Action Input must be RAW python code.
- Format:
Thought: (reasoning)
Action: python_repl_ast
Action Input: your raw python code (strictly no backslashes before underscores)
"""

agent = create_pandas_dataframe_agent(
    llm_with_stop,  # llm,
    df,
    verbose=True,               # Thought/Action/Observation 실시간 출력
    allow_dangerous_code=True,  # 코드 실행 허용
    # 실행 엔진(AgentExecutor)에 전달할 인자를 별도 딕셔너리로 분리
    agent_executor_kwargs={
        # "handle_parsing_errors": "Your output format is wrong. ALWAYS use 'Action: python_repl_ast' and 'Action Input: [code]'. DO NOT use markdown code blocks.",
        "handle_parsing_errors": "Please provide your output in the correct format. Action must be 'python_repl_ast'. Action Input must be RAW python code without any markdown code blocks (no backticks).",
    },
    agent_type="zero-shot-react-description",
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, # 명시적 타입 지정
    max_iterations=20, # 파싱 에러 시 재시도 횟수
    prefix=custom_prefix,
)

# 4. 분석 상세 요구 사항 입력 TODO 테스트용
summary_req = "전체적인 수치 요약해줘"
hypo_req = "대여소(시작_대여소_ID/종료_대여소_ID)별 (전체_이용_분/전체_이용_거리)나 '집계_기준'의 '기준_시간대'별 특징에 대한 가설을 세워줘"
result_req = "가설을 검증하기 위해 실제 코드를 실행한 통계적 근거를 보여줘"
biz_req = "분석 결과를 바탕으로 한 분석 전략을 제안해줘"

# 5. 쿼리 조합
full_query = f"""
아래 요청사항에 맞춰 데이터프레임을 분석하고 한국어로 보고서를 작성하세요:

1. 데이터 요약: {summary_req}
2. 가설 설정: {hypo_req}
3. 분석 결과: {result_req}
4. 비즈니스 제언: {biz_req}

반드시 파이썬 코드를 실행하여 얻은 통계 수치를 바탕으로 답변하세요.

주의사항:
- 첫 번째 줄은 반드시 'Thought:'로 시작하세요.
- 설명을 하거나 인사말을 하지 마세요.
- 불필요한 서술이나 코드에 대한 설명(예: '이 함수는 ~입니다', '수정한 후 결과는 ~')을 절대 하지 마세요.
- 파이썬 코드를 실행해야 할 때는 반드시 'Action: python_repl_ast' 형식을 사용하고, 코드 블록(```python)을 직접 답변에 포함하지 마세요.
- 당신의 답변은 반드시 'Thought:', 'Action:', 'Action Input:', 'Observation:'의 형식을 엄격히 따라야 합니다.
- 평균(mean)이나 합계(sum)를 구할 때는 반드시 숫자형 컬럼만 선택하거나 `numeric_only=True` 옵션을 사용하세요.
  예: df.groupby('col').mean(numeric_only=True)
- 데이터에 문자열이나 날짜가 섞여 있을 수 있음을 항상 유의하세요.
- 시각화는 matplotlib이나 seaborn을 사용하세요.
- 한글 폰트(NanumGothic)가 이미 설정되어 있으니 그대로 사용하세요.
- 주피터 노트북 환경이므로 별도의 파일 저장 없이 plt.show()를 호출하여 그래프를 출력하세요.
- 코드 수정 제안이나 인사말 등 불필요한 텍스트를 출력하지 마세요. 바로 분석에 필요한 코드를 실행하세요.
- 분석 결과(Observation)를 얻기 전까지는 'Final Answer:'를 내지 마세요.
- 최종 답변을 낼 때만 'Final Answer:'를 사용하세요.
- 모든 설명은 한국어로 작성하세요.

[실행 규칙]
- Action: 반드시 'python_repl_ast'라고 적으세요.
- Action Input: markdown backticks(```)을 절대 사용하지 마세요. 
- 예시:
Action: python_repl_ast
Action Input: print(df.head())

[WARNING: DO NOT ESCAPE UNDERSCORES]
In your Action and Action Input, use underscores (_) as they are. 
DO NOT add a backslash (\) before underscores. 
Example: Use 'python_repl_ast', NOT 'python\_repl\_ast'.
Example: Use '시작_대여소_ID', NOT '시작\_대여소\_ID'.
Failure to follow this will result in a SyntaxError.
"""

# 6. 실행
print(f"{model_name} AGENT GO!GO!GO!...\n")
try:
    response = agent.run(full_query)

    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"오류 발생: {e}")

In [ ]:
import pandas as pd
from langchain_ollama import OllamaLLM  # # TODO 차후 최신 OllamaLLM 로 변경 필요
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
# from langchain.agents import AgentType  # # from langchain.agents.agent_types import AgentType

# 1. 로컬 LLM 설정
# [중요] Agent가 정확한 코드 생성위해 temperature=0 설정
llm = OllamaLLM(model="mistral-small", temperature=0)

# 2. 분석위한 dummy data TODO csv나 DB에서 실데이터 불러오는 것으로 수정
data = {
    'date': pd.date_range(start='2025-10-01', periods=14),
    'visitor_count': [1200, 1300, 1100, 900, 850, 1500, 1600, 1150, 1050, 950, 800, 1450, 1550, 1100],
    'order_count': [120, 135, 115, 80, 75, 180, 195, 110, 100, 85, 70, 175, 190, 105],
    'ad_spend': [500, 500, 500, 300, 300, 700, 700, 500, 500, 500, 300, 700, 700, 500],
    'platform': ['PC', 'Mobile', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC', 'Mobile', 'PC', 'PC', 'Mobile', 'Mobile', 'PC']
}
df = pd.DataFrame(data)

# 3. Pandas DataFrame Agent 생성
agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=True, # Agent가 생각 과정(Thought/Action) 화면 출력
    # agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    allow_dangerous_code=True, # # 로컬 환경에서 LLM이 python 실행 허용 설정
)

# 4. 분석 요청 프롬프트 (Few-shot 구조 결합)
query = """
제공된 데이터프레임을 기반으로 다음 구조에 맞춰 한국어 보고서를 작성하세요:

1. 데이터 요약: 전체적인 수치(방문자, 주문수 등) 요약
2. 가설 설정: 플랫폼(PC/Mobile)별 효율 차이나 요일별 특징에 대한 가설
3. 분석 결과: 가설을 검증하기 위해 실제 코드를 실행한 통계적 근거 (예: 플랫폼별 전환율 비교)
4. 비즈니스 제언: 분석 결과를 바탕으로 한 마케팅 전략

반드시 코드를 실행하여 정확한 평균값과 상관관계를 계산한 뒤 답변하세요.
"""

print("Agent GO!GO!GO!...")
print("-" * 50)

try:
    response = agent.run(query)
    print("\n" + "="*50)
    print("최종 분석 보고서")
    print("="*50)
    print(response)
except Exception as e:
    print(f"Agent 실행 중 오류 발생: {e}")